In [ ]:
LEXICON_ALL_DICT = [
    'ae', 'acc', 'ap90', 'armh', 'bor',
    'ben', 'bhs', 'cae', 'gst', 'ieg',
    'inm', 'krm', 'lan', 'mci', 'md',
    'mw', 'mwe', 'mw72', 'pe', 'pui',
    'shs', 'skd', 'snp', 'vcp', 'vei',
    'wil', 'yat', 'pgn', 'eng2te', 'dhatu_pata'
]

# Dictionary - Database Names mapping
LEXICON_ALL_DICT_TO_DB_MAP = (
    (dictName, dictName.upper())
    for dictName in LEXICON_ALL_DICT
)
LEXICON_ALL_DICT_TO_DB_MAP = dict(LEXICON_ALL_DICT_TO_DB_MAP)
LEXICON_ALL_DICT_TO_DB_MAP['eng2te'] = 'ENG2TEL'

# Database - Table Names mapping
LEXICON_ALL_DICT_TO_TABLE_NAMES_MAP = dict(
    (dbName, dbName if dbName != 'dhatu_pata' else 'dictEntries') for dbName in LEXICON_ALL_DICT
)
# Table Names - Word Field mapping
LEXICON_ALL_TABLE_WORD_FIELD_MAP = dict(
    (dbName, 'key') for dbName in LEXICON_ALL_DICT_TO_TABLE_NAMES_MAP.values()
)
LEXICON_ALL_TABLE_WORD_FIELD_MAP['eng2te'] = 'eng_word'
LEXICON_ALL_TABLE_WORD_FIELD_MAP['dhatu_pata'] = 'word'

print(f"Dictionaries ({len(LEXICON_ALL_DICT_TO_DB_MAP)})", LEXICON_ALL_DICT_TO_DB_MAP)
print(f"Database to Table Names ({len(LEXICON_ALL_DICT_TO_TABLE_NAMES_MAP)})", LEXICON_ALL_TABLE_WORD_FIELD_MAP)

In [ ]:
import sqlite3
import pandas as pd

def list_tables(sqlite_file):
  """List all tables in the given SQLite database file."""
  conn = sqlite3.connect(sqlite_file)
  cursor = conn.cursor()
  cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
  tables = [row[0] for row in cursor.fetchall()]
  conn.close()
  return tables

def table_structure(sqlite_file, table_name):
  """Display the structure (columns and types) of the given table."""
  conn = sqlite3.connect(sqlite_file)
  cursor = conn.cursor()
  cursor.execute(f"PRAGMA table_info({table_name});")
  structure = cursor.fetchall()
  conn.close()
  # structure: [(cid, name, type, notnull, dflt_value, pk), ...]
  columns = ["cid", "name", "type", "notnull", "dflt_value", "pk"]
  return pd.DataFrame(structure, columns=columns)

def display_table(sqlite_file, table_name, limit=10):
  """Display the data of the given table as rows and columns (default 10 rows)."""
  conn = sqlite3.connect(sqlite_file)
  df = pd.read_sql_query(f"SELECT * FROM {table_name} LIMIT {limit}", conn)
  conn.close()
  return df

def get_table_rows_count(sqlite_file, table_name):
  """Fetch the number of rows of the given table."""
  conn = sqlite3.connect(sqlite_file)
  cursor = conn.cursor()
  cursor.execute(f"SELECT COUNT(*) FROM {table_name};")
  row_count = cursor.fetchone()[0]
  conn.close()
  return row_count

def get_table_unique_row_count(sqlite_file, table_name, field_name):
  """Fetch the number of unique rows of the given table."""
  conn = sqlite3.connect(sqlite_file)
  cursor = conn.cursor()
  cursor.execute(f"SELECT COUNT(DISTINCT {field_name}) FROM {table_name};")
  unique_row_count = cursor.fetchone()[0]
  conn.close()
  return unique_row_count

# function to return the unique values of the given field
def get_unique_field_values(sqlite_file, table_name, field_name):
    """Fetch unique values of a specific field in the given table."""
    conn = sqlite3.connect(sqlite_file)
    cursor = conn.cursor()
    cursor.execute(f"SELECT DISTINCT {field_name} FROM {table_name};")
    unique_values = [row[0] for row in cursor.fetchall()]
    conn.close()
    return unique_values

In [ ]:
def display_table_with_blob_as_text(sqlite_file, table_name, limit=10):
  conn = sqlite3.connect(sqlite_file)
  cursor = conn.cursor()
  cursor.execute(f"PRAGMA table_info({table_name});")
  columns_info = cursor.fetchall()
  columns = [col[1] for col in columns_info]
  types = {col[1]: col[2].upper() for col in columns_info}

  query = f"SELECT * FROM {table_name} LIMIT {limit}"
  cursor.execute(query)
  rows = cursor.fetchall()
  conn.close()

  # Convert BLOB columns to string
  def convert_blob(val, col):
    if types[col] == 'BLOB' and isinstance(val, (bytes, bytearray)):
      try:
        return val.decode('utf-8', errors='replace')
      except Exception:
        return str(val)
    return val

  processed_rows = [
    [convert_blob(val, col) for val, col in zip(row, columns)]
    for row in rows
  ]
  return pd.DataFrame(processed_rows, columns=columns)

In [ ]:
# function to loop through all dictionaries and count rows
def count_rows_in_all_dictionaries():
    """Count rows in all dictionaries and return a DataFrame with counts."""
    counts = []
    for dict_name, table_name in LEXICON_ALL_DICT_TO_TABLE_NAMES_MAP.items():
        try:
            sqlite_file = f"./tmp/{dict_name}.sqlite"
            row_count = get_table_rows_count(sqlite_file, table_name)
            unique_row_count = get_table_unique_row_count(sqlite_file, table_name, LEXICON_ALL_TABLE_WORD_FIELD_MAP[dict_name])
            counts.append((dict_name, row_count, unique_row_count))
        except sqlite3.OperationalError as e:
            print(f"Error accessing {table_name}: {e}")
            counts.append((dict_name, None, None))

    return pd.DataFrame(counts, columns=['Dictionary', 'Row Count', 'Unique Row Count'])

def count_unique_words_in_all_dictionaries():
    """Count unique words in all dictionaries and return a DataFrame with counts."""
    words = set()
    for dict_name, table_name in LEXICON_ALL_DICT_TO_TABLE_NAMES_MAP.items():
        try:
            sqlite_file = f"./tmp/{dict_name}.sqlite"
            unique_words = get_unique_field_values(sqlite_file, table_name, LEXICON_ALL_TABLE_WORD_FIELD_MAP[dict_name])
            words.update(unique_words)
        except sqlite3.OperationalError as e:
            print(f"Error accessing {table_name}: {e}")

    # remove duplicates and return the count
    return len(words)


In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
# Count rows in all dictionaries and display totals
# check Readme.md for the list of dictionaries and last calcualted totals
def calculate_totals():
  totals_per_each_db = count_rows_in_all_dictionaries()
  total_unique_words = count_unique_words_in_all_dictionaries()
  totals = totals_per_each_db['Row Count'].sum()
  unique_totals = totals_per_each_db['Unique Row Count'].sum()
  print("records:", totals, "unique words (individual):", unique_totals, "total unique words:", total_unique_words)
  print(totals_per_each_db.sort_values('Unique Row Count', ascending=False))

# calculate_totals()

In [ ]:
# sqlite_file = "./tmp/eng2te.sqlite"
# sqlite_file = "./tmp/dhatu_pata.sqlite"
sqlite_file = "./tmp/mw.sqlite"

In [ ]:
list_tables(sqlite_file)

In [ ]:
# table_structure(sqlite_file, table_name)
for tbl in list_tables(sqlite_file):
  print(f"Table: {tbl}")
  display(table_structure(sqlite_file, tbl))

In [ ]:
# display_table(sqlite_file, table_name, limit=10)
# display_table_with_blob_as_text(sqlite_file, table_name, limit=10)
for tbl in list_tables(sqlite_file):
  print(f"Table: {tbl} - Rows: {get_table_rows_count(sqlite_file, tbl)}")
  display(display_table_with_blob_as_text(sqlite_file, tbl, limit=50))

In [ ]:
table_name = 'dictEntries_segdir'

In [ ]:
# display_table(sqlite_file, "dictAbbs", limit=50)